In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

pd.set_option('display.max_column', 500)
pd.set_option('display.max_rows', 5000)
random.seed(1001)
sns.set(font="Arial Unicode MS")

### Data Preprocessing

In [2]:
dftrain = pd.read_csv('X_train.csv' , index_col=0)
dftest = pd.read_csv('X_test.csv', index_col=0)
y_train_all = pd.read_csv('y_train.csv', index_col=0)

print(f"Training df shape = {dftrain.shape}; test df shape = {dftest.shape}")

Training df shape = (9460, 41); test df shape = (2366, 41)


In [3]:
# 把 建築完成年月 換算成 建築年齡
dftrain['建築年齡'] = dftrain['建築完成年月'].apply(lambda x: 
                        (datetime.now() - datetime.strptime(x, "%Y-%m-%d")).days / 365.25)

# 丟掉 建築完成年月
dftrain.drop(['建築完成年月'], axis=1, inplace=True)

dftest['建築年齡'] = dftest['建築完成年月'].apply(lambda x:
                        (datetime.now() - datetime.strptime(x, "%Y-%m-%d")).days / 365.25)

dftest.drop(['建築完成年月'], axis=1, inplace=True)

In [4]:
# 把 交易年	交易日 交易月 丟掉，變成有小數點的年份
dftrain['交易年'] = dftrain['交易年'] + dftrain['交易月'] / 12 + dftrain['交易日'] / 365.25
dftrain.drop(['交易日', '交易月'], axis=1, inplace=True)

dftest['交易年'] = dftest['交易年'] + dftest['交易月'] / 12 + dftest['交易日'] / 365.25
dftest.drop(['交易日', '交易月'], axis=1, inplace=True)

In [5]:
# 交易年 - min()
mini = dftrain['交易年'].min()
dftrain['交易年'] = dftrain['交易年'] - mini
dftest['交易年'] = dftest['交易年'] - mini

In [6]:
dftrain.head()

,鄉鎮市區,交易標的,路名,土地移轉總面積平方公尺,都市土地使用分區,土地數,建物數,車位數,移轉層次,移轉層次項目,總樓層數,建物型態,主要用途,主要建材,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,交易年,地鐵站,超商,公園,托兒所,國小,國中,高中職,大學,金融機構,醫院,大賣場,超市,百貨公司,警察局,消防局,縱坐標,橫坐標,建築年齡
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,文山區,房地(土地+建物)+車位,興隆路三段,27.75,住,1.0,1.0,1.0,1,無,7.0,華廈(10層含以下有電梯),見其他登記事項,鋼筋混凝土造,133.43,3,2,2,有,有,7.382615,1.0,7.0,2.0,20.0,20.0,19.0,12.0,17.0,15.0,20.0,7.0,20.0,13.0,20.0,16.0,24.957269,121.588026,5.100616
1,中正區,房地(土地+建物),金山南路一段,9.57,第三種住宅區,1.0,1.0,0.0,5,無,6.0,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,40.34,1,1,1,有,有,8.736311,1.0,12.0,8.0,20.0,20.0,20.0,13.0,20.0,20.0,20.0,13.0,20.0,20.0,20.0,20.0,24.997141,121.558262,27.255305
2,文山區,房地(土地+建物),秀明路一段,9.51,住,1.0,1.0,0.0,1,無,7.0,套房(1房1廳1衛),住家用,鋼筋混凝土造,70.61,1,1,1,有,有,8.043806,0.0,6.0,7.0,20.0,20.0,18.0,11.0,19.0,15.0,18.0,6.0,20.0,11.0,20.0,13.0,24.953906,121.601050,15.526352
3,內湖區,房地(土地+建物)+車位,康樂街,23.67,第三種住宅區,1.0,1.0,1.0,10,無,15.0,住宅大樓(11層含以上有電梯),見其他登記事項,鋼筋混凝土造,143.83,3,2,2,有,有,7.144764,1.0,12.0,14.0,20.0,20.0,14.0,5.0,4.0,19.0,20.0,12.0,20.0,13.0,20.0,15.0,25.008046,121.557424,15.578371
4,北投區,房地(土地+建物),公路,22.50,第三種住宅區,1.0,1.0,0.0,2,無,5.0,公寓(5樓含以下無電梯),住家用,鋼筋混凝土造,83.73,2,1,1,有,無,8.986311,2.0,4.0,13.0,20.0,20.0,15.0,5.0,15.0,19.0,20.0,10.0,20.0,16.0,18.0,13.0,24.986825,121.557424,51.764545


### Generate Features

In [7]:
def gen_features(dftrain, dftest, numfeatures, catfeatures, encoder):
    # Combine train and test data for categorical features
    combined_cat = pd.concat([dftrain[catfeatures], dftest[catfeatures]])
    
    # Fit encoder on the combined data
    encoder.fit(combined_cat)
    
    # Transform training data
    X_train = dftrain[numfeatures].to_numpy()  # Numerical features
    X_train_cat = encoder.transform(dftrain[catfeatures]).toarray()  # Encoded categorical features
    X_train = np.hstack((X_train, X_train_cat))
    
    # Transform testing data
    X_test = dftest[numfeatures].to_numpy()  # Numerical features
    X_test_cat = encoder.transform(dftest[catfeatures]).toarray()  # Encoded categorical features
    X_test = np.hstack((X_test, X_test_cat))
    
    return X_train, X_test

In [8]:
#get the column names of dftrain
numfeatures = dftrain.select_dtypes(include=[np.number]).columns
print(f"Numerical features = {numfeatures}")
print(f"Number of numerical features = {len(numfeatures)}")
numfeatures = numfeatures.tolist()

# Update np.object to just 'object' in the dtype selection
catfeatures = dftrain.select_dtypes(include=['object']).columns
print(f"Categorical features = {catfeatures}")
print(f"Number of categorical features = {len(catfeatures)}")
catfeatures = catfeatures.tolist()

Numerical features = Index(['土地移轉總面積平方公尺', '土地數', '建物數', '車位數', '移轉層次', '總樓層數', '建物移轉總面積平方公尺',
       '建物現況格局-房', '建物現況格局-廳', '建物現況格局-衛', '交易年', '地鐵站', '超商', '公園', '托兒所',
       '國小', '國中', '高中職', '大學', '金融機構', '醫院', '大賣場', '超市', '百貨公司', '警察局',
       '消防局', '縱坐標', '橫坐標', '建築年齡'],
      dtype='object')
Number of numerical features = 29
Categorical features = Index(['鄉鎮市區', '交易標的', '路名', '都市土地使用分區', '移轉層次項目', '建物型態', '主要用途', '主要建材',
       '建物現況格局-隔間', '有無管理組織'],
      dtype='object')
Number of categorical features = 10


In [9]:
X_train_all, X_test_all = gen_features(dftrain, dftest, numfeatures, catfeatures, OneHotEncoder())

print(f"X_train shape = {X_train_all.shape}; X_test shape = {X_test_all.shape}")

X_train shape = (9460, 741); X_test shape = (2366, 741)


### Load the trained model and get the result.
The result is saved in the_final_prediction.csv

In [10]:
import pickle as pkl

In [11]:
with open('final_model_xgboost.pkl', 'rb') as f:
    final_model_xgboost = pkl.load(f)
with open('final_model_gdboost.pkl', 'rb') as f:
    final_model_gdboost = pkl.load(f)

In [ ]:
y_pred_xgboost = final_model_xgboost.predict(X_test_all)
y_pred_gdboost = final_model_gdboost.predict(X_test_all)

#average the two models
y_pred = (y_pred_xgboost + y_pred_gdboost) / 2

#write to another csv file
y_pred_df = pd.DataFrame(y_pred, columns=['單價元平方公尺'])
y_pred_df.to_csv('final_pred.csv')